# ShapeNet-Airplane Attention Visualization
- By running this script, you can get images of attention-based-color-coded pointclouds.
## To run this code...
- You should prepare the summary file by running sample_and_summarize.py with a trained checkpoint.
- You should install below libraries.
    - matplotlib
    - open3d
    - numpy
    - torch
    - torchvision
    - tqdm

In [37]:
import os
import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#import open3d as o3d

import numpy as np
import torch
torch.no_grad()
from torchvision.utils import save_image, make_grid

from draw import draw, draw_attention_pointcloud

## Set directories
1. experiment_name: log_name.lstrip('gen/') in scripts
2. save_dir: path to save images

In [38]:
save_dir = 'images_attn'
experiment_name = 'shapenet15k-airplane/n_samples_2000/'
summary_name = os.path.join('/data/rna_rep_learning/nmsingh/scSet_ckpts/sample_complexity/', experiment_name, 'summary.pth')
summary_train_name = os.path.join('/data/rna_rep_learning/nmsingh/scSet_ckpts/sample_complexity/', experiment_name, 'summary_train_recon.pth')

imgdir = os.path.join(save_dir, experiment_name)
imgdir_recon = os.path.join(imgdir, 'recon')
imgdir_gt = os.path.join(imgdir, 'gt')
imgdir_gen = os.path.join(imgdir, 'gen')
imgdir_gt_train = os.path.join(imgdir, 'gt_train')

os.makedirs(save_dir, exist_ok=True)
os.makedirs(imgdir_gt, exist_ok=True)
os.makedirs(imgdir_recon, exist_ok=True)
os.makedirs(imgdir_gen, exist_ok=True)
os.makedirs(imgdir_gt_train, exist_ok=True)

In [39]:
# validation and generated
summary = torch.load(summary_name)
for k, v in summary.items():
    try:
        print(f"{k}: {v.shape}")
    except AttributeError:
        print(f"{k}: {len(v)}")
len_att = len(summary['dec_att'])

smp_set: torch.Size([405, 2500, 3])
smp_mask: torch.Size([405, 2500])
smp_att: 7
priors: 8
recon_set: torch.Size([405, 2500, 3])
recon_mask: torch.Size([405, 2500])
posteriors: 8
dec_att: 7
enc_att: 7
enc_hiddens: 13
gt_set: torch.Size([405, 2048, 3])
gt_mask: torch.Size([405, 2048])
mean: torch.Size([405, 1, 3])
std: torch.Size([405, 1, 1])
sid: 13
mid: 13
pid: 13
cardinality: 13


## Select the samples to visualize
- parse the samples by index.
- below default code will visualize all samples. **Warning: Requires Huge Memory**

In [40]:
n_viz=10
recon_targets = list(range(len(summary['gt_mask'])))[:n_viz]
gen_targets = list(range(len(summary['smp_mask'])))[:n_viz]

gt = summary['gt_set'][recon_targets]
gt_mask = summary['gt_mask'][recon_targets]

recon = summary['recon_set'][recon_targets]
recon_mask = summary['recon_mask'][recon_targets]

dec_att = [summary['dec_att'][l][:, :, recon_targets] for l in range(len_att)]
enc_att = [summary['enc_att'][l][:, :, recon_targets] for l in range(len_att)]

gen = summary['smp_set'][gen_targets]
gen_mask = summary['smp_mask'][gen_targets]
gen_att = [summary['smp_att'][l][:, :, gen_targets] for l in range(len_att)]

## Visualize Attention
- lidx: index of layer
- projection: ISAB has 2 projection attention and back-projection attention.
    - 0: projection, 1: back-projection

In [41]:
def attention_selector(gt, gt_mask, att, lidx=0, projection=0, selected_heads=None, palette_permutation=None):
    if selected_heads is not None:
        att = [a[:, selected_heads].view(a.size(0), len(selected_heads), a.size(2), a.size(3), a.size(4)) for a in att]
    return draw_attention_pointcloud(gt, gt_mask, att[lidx][projection], color_opt='gist_rainbow')

### Visualize Encoder Attention on GT samples

In [42]:
for topdown in tqdm(range(2,5)):
    for projection in [0]:
        if topdown == 2:
            selected_heads = [2]
            palette_permutation = [1, 0]
        elif topdown == 3:
            selected_heads = [0]
            palette_permutation = [1, 2, 3, 0]
        elif topdown == 4:
            selected_heads = [2]
            palette_permutation = None
        else:
            selected_heads = list(range(enc_att[0].size(1)))
            palette_permutation = None
        print(f"HEAD: {selected_heads}, COLOR: {palette_permutation}")
        gt_imgs = attention_selector(gt, gt_mask, enc_att, len(enc_att) - 1 - topdown, projection,
                                     selected_heads=selected_heads, palette_permutation=palette_permutation)
        for head_idx in range(len(selected_heads)):
            for idx in range(len(recon_targets)):
                data_idx = recon_targets[idx]
                try:
                    pos_min = torch.nonzero(gt_imgs[idx][head_idx].mean(0) != 1).min(0)[0]
                    pos_max = torch.nonzero(gt_imgs[idx][head_idx].mean(0) != 1).max(0)[0]
                    gt_img = gt_imgs[idx][head_idx][:, pos_min[0]:pos_max[0]+1, pos_min[1]:pos_max[1]+1]
                except RuntimeError:
                    gt_img = gt_imgs[idx][head_idx].float()
                head = selected_heads[head_idx]
                save_image(gt_img, os.path.join(imgdir_gt, f'{topdown}_{projection}_{head}_{data_idx}.png'))
        del gt_imgs
print('gt DONE')

HEAD: [2], COLOR: [1, 0]
HEAD: [0], COLOR: [1, 2, 3, 0]
HEAD: [2], COLOR: None

gt DONE


### Visualize Decoder Attention on Generated samples

In [43]:
for topdown in tqdm(range(2, 5)):
    for projection in [1]:
        if topdown == 2:
            selected_heads = [1]
            palette_permutation = [0, 1]
        elif topdown == 3:
            selected_heads = [0, 2]
            palette_permutation = [0, 1, 2, 3]
        elif topdown == 4:
            selected_heads = [0, 1, 3]
            palette_permutation = None
        else:
            selected_heads = list(range(gen_att[0].size(1)))
            palette_permutation = None
        print(f"HEAD: {selected_heads}, COLOR: {palette_permutation}")
        gen_imgs = attention_selector(gen, gen_mask, gen_att, topdown, projection, selected_heads, palette_permutation)
        for head_idx in range(len(selected_heads)):
            for idx in range(len(gen_targets)):
                data_idx = gen_targets[idx]
                try:
                    pos_min = torch.nonzero(gen_imgs[idx][head_idx].mean(0) != 1).min(0)[0]
                    pos_max = torch.nonzero(gen_imgs[idx][head_idx].mean(0) != 1).max(0)[0]
                    gen_img = gen_imgs[idx][head_idx][:, pos_min[0]:pos_max[0]+1, pos_min[1]:pos_max[1]+1]
                except RuntimeError:
                    gen_img = gen_imgs[idx][head_idx]
                head = selected_heads[head_idx]
                save_image(gen_img.float(), os.path.join(imgdir_gen, f'{topdown}_{projection}_{head}_{data_idx}.png'))
        del gen_imgs
print('gen DONE')

HEAD: [1], COLOR: [0, 1]
HEAD: [0, 2], COLOR: [0, 1, 2, 3]
HEAD: [0, 1, 3], COLOR: None

gen DONE
